In [1]:
import torch
import torch.nn.functional as F
import torch.optim as optim

from dhg.data import Cora
from dhg import Graph, Hypergraph
from dhg.models import HGNN
from dhg.metrics import HypergraphVertexClassificationEvaluator as Evaluator
from dhg.random import set_seed

import time
from copy import deepcopy

In [2]:
def train(A, X, lbls, net, train_idx, optimizer, epoch):

    net.train()

    st = time.time()
    optimizer.zero_grad()
    outs = net(X, A)
    outs, lbls = outs[train_idx], lbls[train_idx]
    loss = F.cross_entropy(outs, lbls)
    loss.backward()
    optimizer.step()

    print(f"Epoch: {epoch}, Time: {time.time()-st:.5f}s, Loss: {loss.item():.5f}")

    return loss.item()

In [3]:
@torch.no_grad()
def infer(A, X, lbls, net, idx, test=False):

    net.eval()

    outs = net(X, A)
    outs, lbls = outs[idx], lbls[idx]

    if not test:
        res = evaluator.validate(lbls, outs)
    else:
        res = evaluator.test(lbls, outs)

    return res

In [4]:
if __name__ == "__main__":

    set_seed(2022)

    device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

    ################################################################
    data = Cora()
    G = Graph(data["num_vertices"], data["edge_list"])
    HG = Hypergraph.from_graph_kHop(G, k=1)
    X, lbl = data["features"], data["labels"]

    train_mask = data["train_mask"]
    val_mask   = data["val_mask"]
    test_mask  = data["test_mask"]

    net = HGNN(data["dim_features"], 16, data["num_classes"])
    optimizer = optim.Adam(net.parameters(), lr=0.01, weight_decay=5e-4)

    evaluator = Evaluator(["accuracy", "f1_score", {"f1_score": {"average": "micro"}}])

    ################################################################
    HG = HG.to(device)
    X, lbl = X.to(device), lbl.to(device)
    net = net.to(device)

    ################################################################
    best_val = 0
    best_state = None
    best_epoch = 0
    for epoch in range(200):

        # train, 每轮optimizer的参数在优化
        train(HG, X, lbl, net, train_mask, optimizer, epoch)

        # validation
        if epoch % 1 == 0:
            with torch.no_grad():
                val_res = infer(HG, X, lbl, net, val_mask)
            if val_res > best_val:
                print(f"update best: {val_res:.5f}")
                best_val   = val_res
                best_state = deepcopy(net.state_dict())
                best_epoch = epoch

    print("\ntrain finished!")
    print(f"best val: {best_val:.5f}")

    # test
    print("test...")
    net.load_state_dict(best_state)
    res = infer(HG, X, lbl, net, test_mask, test=True)
    print(f"final result: epoch: {best_epoch}")
    print(res)

Epoch: 0, Time: 0.77317s, Loss: 1.94993
update best: 0.31600
Epoch: 1, Time: 0.00156s, Loss: 1.94627
Epoch: 2, Time: 0.00125s, Loss: 1.94413
Epoch: 3, Time: 0.00119s, Loss: 1.93941
Epoch: 4, Time: 0.00126s, Loss: 1.93488
Epoch: 5, Time: 0.00108s, Loss: 1.92980
update best: 0.32000
Epoch: 6, Time: 0.00105s, Loss: 1.92559
update best: 0.36400
Epoch: 7, Time: 0.00098s, Loss: 1.91934
update best: 0.46400
Epoch: 8, Time: 0.00096s, Loss: 1.91385
Epoch: 9, Time: 0.00091s, Loss: 1.90319
Epoch: 10, Time: 0.00100s, Loss: 1.89834
Epoch: 11, Time: 0.00084s, Loss: 1.89127
Epoch: 12, Time: 0.00084s, Loss: 1.87880
Epoch: 13, Time: 0.00085s, Loss: 1.87467
Epoch: 14, Time: 0.00084s, Loss: 1.86688
Epoch: 15, Time: 0.00082s, Loss: 1.85891
Epoch: 16, Time: 0.00079s, Loss: 1.85094
Epoch: 17, Time: 0.00091s, Loss: 1.84161
Epoch: 18, Time: 0.00074s, Loss: 1.82744
Epoch: 19, Time: 0.00077s, Loss: 1.81666
Epoch: 20, Time: 0.00074s, Loss: 1.80902
Epoch: 21, Time: 0.00073s, Loss: 1.78956
Epoch: 22, Time: 0.00072